In [ ]:
import arcpy
import geojson
import json
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import os
import time
import numpy as np
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, MultiPolygon, shape
from pathlib import Path
import pathlib


PLANET_API_KEY=''
auth = HTTPBasicAuth(PLANET_API_KEY, '')

#Directory paths
# bDir = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\'
# iDir = bDir + 'json'
# oDir = bDir + 'images'

out_folder = 'C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\images\\All_Sensor_Courses\\'
file_path = "C:\\Users\\msmurphy\\Documents\\WinterTurf Planet\\Sensor Courses\\Bulk PS2SD Output.csv"

print(datetime.now())


2025-09-15 08:26:27.595877


In [5]:
def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        # timer the reason this failed last time
        # absneeds more than 10 seconds
#         time.sleep(10)
        time.sleep(60)
        
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join(oDir, n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))

In [7]:

df = pd.read_csv(file_path)

base_url = 'https://api.planet.com/compute/ops/orders/v2/'

# years = ['2017','2018','2019','2020','2021','2022','2023','2024','2025']
years = ['2019','2020','2021','2022']
# years = ['2024']

for year in years:
    for index, row in df.iterrows(): 
        # order_url = base_url + row['Order_'+year] 
        order_url = row['Order_'+year]
        
        course_name = row['Course']

        oDir = out_folder + course_name

        if not os.path.exists(oDir):
            os.makedirs(oDir)

        downloaded_clip_files = download_order(order_url, auth)

        clip_img_file = next(downloaded_clip_files[d] for d in downloaded_clip_files
                            if d.endswith('_clip.tif'))

<Response [200]>
25 items to download
C:\Users\msmurphy\Documents\WinterTurf Planet\Sensor Courses\images\All_Sensor_Courses\Course_01M_Outline\03f2abec-6530-4c48-9445-474288b8f542\PSScene\20190820_162041_16_1068_3B_AnalyticMS_metadata_clip.xml already exists, skipping 03f2abec-6530-4c48-9445-474288b8f542/PSScene/20190820_162041_16_1068_3B_AnalyticMS_metadata_clip.xml
C:\Users\msmurphy\Documents\WinterTurf Planet\Sensor Courses\images\All_Sensor_Courses\Course_01M_Outline\03f2abec-6530-4c48-9445-474288b8f542\PSScene\20190820_162041_16_1068_metadata.json already exists, skipping 03f2abec-6530-4c48-9445-474288b8f542/PSScene/20190820_162041_16_1068_metadata.json
C:\Users\msmurphy\Documents\WinterTurf Planet\Sensor Courses\images\All_Sensor_Courses\Course_01M_Outline\03f2abec-6530-4c48-9445-474288b8f542\PSScene\20190820_162041_16_1068_3B_udm2_clip.tif already exists, skipping 03f2abec-6530-4c48-9445-474288b8f542/PSScene/20190820_162041_16_1068_3B_udm2_clip.tif
C:\Users\msmurphy\Documents\W

MissingSchema: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?